In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.datasets import make_classification
%matplotlib inline

In [4]:
df = pd.read_csv('EventForML.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468962 entries, 0 to 468961
Data columns (total 15 columns):
hour           468962 non-null int64
minute         468962 non-null int64
TotalTime      468962 non-null float64
CauseType      468962 non-null float64
Region         468962 non-null int64
PEALv          468962 non-null int64
Weather        468962 non-null int64
Cus            468962 non-null float64
day            468962 non-null int64
month          468962 non-null int64
year           468962 non-null int64
Date           468962 non-null int64
EquipWork_B    468962 non-null int64
EquipWork_F    468962 non-null int64
EquipWork_R    468962 non-null int64
dtypes: float64(3), int64(12)
memory usage: 53.7 MB


In [5]:
df.head()

,hour,minute,TotalTime,CauseType,Region,PEALv,Weather,Cus,day,month,year,Date,EquipWork_B,EquipWork_F,EquipWork_R
0,0,30,60.0,5.0,2,1,1,2.314815e-08,1,1,2013,3,0,1,0
1,0,30,15.0,7.0,1,1,2,4.629630e-11,1,1,2013,3,0,1,0
2,1,0,60.0,7.0,1,1,1,9.074074e-09,1,1,2013,3,0,1,0
3,1,30,50.0,5.0,11,1,2,4.629630e-11,1,1,2013,3,0,1,0
4,1,50,40.0,7.0,1,1,1,1.157407e-08,1,1,2013,3,0,1,0


In [6]:
df = df.loc[df['Region'] == 7] 

In [7]:
df = df.drop(['Region', 'Cus','TotalTime','year'], axis=1)

In [8]:
df.head()

,hour,minute,CauseType,PEALv,Weather,day,month,Date,EquipWork_B,EquipWork_F,EquipWork_R
28,7,15,5.0,1,2,1,1,3,0,1,0
51,9,30,5.0,1,1,1,1,3,0,1,0
53,9,35,7.0,1,1,1,1,3,0,1,0
59,10,0,5.0,1,1,1,1,3,0,1,0
117,15,7,7.0,2,1,1,1,3,1,0,0


In [9]:
X = df.drop(['CauseType'], axis=1).values #.values คือการเปลี่ยน df เป็น array เรากำลังทำไปเพื่อสร้าง model
y= df['CauseType'].values

In [10]:
y_label = df["CauseType"].values

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, random_state =2019)

In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((22311, 10), (9562, 10), (22311,), (9562,))

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
clf_rfc = RandomForestClassifier() #clf = classifaication, rfc = RandomForestClassifier

In [16]:
%%time
clf_rfc.fit(X_train, y_train)

CPU times: user 1.8 s, sys: 56.1 ms, total: 1.86 s
Wall time: 1.86 s


RandomForestClassifier()

In [17]:
y_pred = clf_rfc.predict(X_test)
y_pred[:5]

array([5., 8., 8., 8., 5.])

In [18]:
y_test[:5]

array([5., 2., 5., 8., 5.])

In [19]:
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support

In [20]:
set (y_label) #ดูว่า y ที่จะทำนายมีกี่ class

{1.0, 2.0, 5.0, 7.0, 8.0, 9.0}

In [21]:
class_names = list(set(y_label))
class_names = [str(x) for x in class_names]
class_names 

['1.0', '2.0', '5.0', '7.0', '8.0', '9.0']

In [22]:
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(data=cm, columns=class_names, index=class_names)
df_cm.style.background_gradient()

,1.0,2.0,5.0,7.0,8.0,9.0
1.0,530,156,338,6,104,91
2.0,163,1002,598,4,122,51
5.0,135,338,3783,0,120,31
7.0,22,15,33,3,6,7
8.0,84,56,76,0,1046,82
9.0,128,48,35,1,176,172


In [23]:
from sklearn.metrics import classification_report

In [24]:
predictions = clf_rfc.predict(X_test)

In [25]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         1.0       0.50      0.43      0.46      1225
         2.0       0.62      0.52      0.56      1940
         5.0       0.78      0.86      0.82      4407
         7.0       0.21      0.03      0.06        86
         8.0       0.66      0.78      0.72      1344
         9.0       0.40      0.31      0.35       560

    accuracy                           0.68      9562
   macro avg       0.53      0.49      0.49      9562
weighted avg       0.67      0.68      0.67      9562



In [26]:
pre, rec, f1, _ = precision_recall_fscore_support(y_test,y_pred)

In [27]:
for idx, c_name in enumerate(class_names): 
    print("CauseType {}: precision: {:.3f}  rec: {:.3f}  f1: {:.3f}".format(c_name,
                                                   pre[idx]*100,
                                                   rec[idx]*100,
                                                   f1[idx]*100))

CauseType 1.0: precision: 49.906  rec: 43.265  f1: 46.349
CauseType 2.0: precision: 62.043  rec: 51.649  f1: 56.371
CauseType 5.0: precision: 77.791  rec: 85.841  f1: 81.618
CauseType 7.0: precision: 21.429  rec: 3.488  f1: 6.000
CauseType 8.0: precision: 66.455  rec: 77.827  f1: 71.693
CauseType 9.0: precision: 39.631  rec: 30.714  f1: 34.608


In [28]:
col_feature = list(df.columns)
df[col_feature] = \
    df[col_feature].astype("float")
col_feature

['hour',
 'minute',
 'CauseType',
 'PEALv',
 'Weather',
 'day',
 'month',
 'Date',
 'EquipWork_B',
 'EquipWork_F',
 'EquipWork_R']

In [29]:
col_feature = [x for x in col_feature if x != 'CauseType']

In [30]:
fi_ls = clf_rfc.feature_importances_

In [31]:
len(fi_ls), len(col_feature)

(10, 10)

In [32]:
df_fi = pd.DataFrame(data=fi_ls, index=col_feature) \
    .rename(columns={0:"feature_importance"})
df_fi = df_fi.sort_values(by="feature_importance",
                         ascending=False)

In [33]:
df_fi[:10].style.bar()

,feature_importance
Weather,0.245962
day,0.175703
hour,0.138721
minute,0.124867
month,0.116619
Date,0.0952372
EquipWork_F,0.0445991
EquipWork_B,0.0405875
PEALv,0.016331
EquipWork_R,0.00137289


In [42]:
import csv
y_prop = clf_rfc.predict_proba(X_test)
zippedList =  list(zip(y_test, y_pred))
result = pd.DataFrame(zippedList, columns = ['สาเหตุจริง', 'สาเหตุพยากรณ์'])
prop = pd.DataFrame(y_prop, columns = ['ต้นไม้','อุปกรณ์','สัตว์','วัสดุแปลกปลอม','สภาพแวดล้อม','ภัยธรรมชาติ'])
prop = prop.round(decimals = 2)
prop['Max_Prop'] = prop.max(axis=1)
input_predict = pd.DataFrame(X_test, columns = col_feature)
predict_result = pd.concat([input_predict,result,prop],axis=1) 
predict_result.to_csv('probs_Xtest_C1.csv',encoding='utf-8',quoting=csv.QUOTE_NONE,index = False)

In [43]:
predict_result.head()

,hour,minute,PEALv,Weather,day,month,Date,EquipWork_B,EquipWork_F,EquipWork_R,สาเหตุจริง,สาเหตุพยากรณ์,ต้นไม้,อุปกรณ์,สัตว์,วัสดุแปลกปลอม,สภาพแวดล้อม,ภัยธรรมชาติ,Max_Prop
0,5,40,1,1,26,8,3,0,1,0,5.0,5.0,0.09,0.06,0.85,0.0,0.00,0.00,0.85
1,16,24,2,6,17,5,7,0,1,0,2.0,8.0,0.04,0.00,0.18,0.0,0.59,0.19,0.59
2,19,0,2,6,20,8,2,0,1,0,5.0,8.0,0.28,0.00,0.07,0.0,0.42,0.23,0.42
3,19,0,1,6,16,4,3,0,1,0,8.0,8.0,0.00,0.02,0.04,0.0,0.87,0.07,0.87
4,6,50,1,1,5,7,6,0,1,0,5.0,5.0,0.15,0.05,0.79,0.0,0.00,0.01,0.79


In [128]:
rfc_predict = clf_rfc.predict_proba(X_train)
rfc_predict[:5]

array([[0.12, 0.11, 0.08, 0.01, 0.  , 0.68],
       [0.  , 0.83, 0.17, 0.  , 0.  , 0.  ],
       [0.82, 0.02, 0.01, 0.  , 0.08, 0.07],
       [0.01, 0.93, 0.06, 0.  , 0.  , 0.  ],
       [0.13, 0.72, 0.05, 0.01, 0.06, 0.03]])

In [129]:
# Evaluate the model's accuracy
print("For Random Forest model:")
print("Train set accuracy = " + str(clf_rfc.score(X_train, y_train)))
print("Test set accuracy = " + str(clf_rfc.score(X_test, y_test)))

For Random Forest model:
Train set accuracy = 0.994262919636054
Test set accuracy = 0.6807153315206024


In [130]:
import pickle
# save the model to disk
filename = 'C1_Ratio7Y_RFC1.sav'
pickle.dump(clf_rfc, open(filename, 'wb'))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

In [ ]:
rfc = RandomForestClassifier(random_state=2019,n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True)

In [ ]:
param_grid = { 
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2'],
}

In [ ]:
grid_obj = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)

In [ ]:
grid_fit = grid_obj.fit(X, y)

In [ ]:
best_rfc_clf = grid_fit.best_estimator_

In [ ]:
%%time
best_rfc_clf.fit(X_train, y_train)

In [ ]:
y_pred = best_rfc_clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(data=cm, columns=class_names, index=class_names)
df_cm.style.background_gradient()

In [ ]:
best_predictions = best_rfc_clf.predict(X_test)

In [ ]:
print(classification_report(y_test,best_predictions))

In [ ]:
pre, rec, f1, _ = precision_recall_fscore_support(y_test,y_pred)

In [ ]:
for idx, c_name in enumerate(class_names): 
    print("CauseType {}: precision: {:.3f}  rec: {:.3f}  f1: {:.3f}".format(c_name,
                                                   pre[idx]*100,
                                                   rec[idx]*100,
                                                   f1[idx]*100))

In [ ]:
fi_ls = best_rfc_clf.feature_importances_

In [ ]:
df_fi = pd.DataFrame(data=fi_ls, index=col_feature) \
    .rename(columns={0:"feature_importance"})
df_fi = df_fi.sort_values(by="feature_importance",
                         ascending=False)

In [ ]:
df_fi[:10].style.bar()

In [ ]:
# Evaluate the model's accuracy
print("For Random Forest model:")
print("Train set accuracy = " + str(best_rfc_clf.score(X_train, y_train)))
print("Test set accuracy = " + str(best_rfc_clf.score(X_test, y_test)))

In [ ]:
X= df[['Weather','TotalTime','Ratio','Cus','hour','minute','day','month']].values

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, random_state =2019)

In [ ]:
grid_obj = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
grid_fit = grid_obj.fit(X, y)
best_rfc_clf = grid_fit.best_estimator_

In [ ]:
%%time
best_rfc_clf.fit(X, y)

In [ ]:
y_pred = best_rfc_clf.predict(X_test)

In [ ]:
best_predictions = best_rfc_clf.predict(X_test)

In [ ]:
pre, rec, f1, _ = precision_recall_fscore_support(y_test,y_pred)

In [ ]:
df2 = df[['Weather','TotalTime','Ratio','Cus','hour','minute','day','month']]
col_feature2 = list(df2.columns)
df[col_feature2] = \
    df[col_feature2].astype("float")
col_feature2

In [ ]:
fi_ls = best_rfc_clf.feature_importances_
df_fi = pd.DataFrame(data=fi_ls, index=col_feature2) \
    .rename(columns={0:"feature_importance"})
df_fi = df_fi.sort_values(by="feature_importance",
                         ascending=False)
df_fi[:10].style.bar()

In [ ]:
best_pred_cutting = best_rfc_clf.predict(X_test)

In [ ]:
print(classification_report(y_test,best_pred_cutting))

In [ ]:
y_prop = best_rfc_clf.predict_proba(X_train)

In [ ]:
y_prop = y_prop.reshape((y_prop.shape[0],y_prop.shape[1]))

In [ ]:
y_prop = pd.DataFrame(y_prop, columns = ['5', '1','2','8','9','7'])

In [ ]:
y_prop

In [ ]:
zippedList =  list(zip(y_test, y_pred))

In [ ]:
result = pd.DataFrame(zippedList, columns = ['Cause_before', 'Cause_after'])

In [ ]:
predict_result = pd.concat([result,y_prop],axis=1) 
predict_result.head()

In [ ]:
import pickle
# save the model to disk
filename = 'C1_Ratio7Y_RFC.sav'
pickle.dump(best_rfc_clf, open(filename, 'wb'))

# Sectionnnnnnnn


In [ ]:
from sklearn.tree import export_graphviz
import pydot
export_graphviz(best_rfc_clf.estimators_[0], out_file='tree_from_forest.dot')
(graph,) = pydot.graph_from_dot_file('tree_from_forest.dot')
graph.write_png('tree_from_forest.png')

In [ ]:
import graphviz
with open("tree_from_forest.dot") as f:
    dot_graph=f.read()
graphviz.Source(dot_graph)

In [ ]:
# graph = pydot.graph_from_dot_data(dot_graph)
graph.write_pdf("tree_from_forest.pdf")

# Machine Learning Algorithm Performance Metrics

# Evaluate the Performance of Machine

In [ ]:
# Evaluate using a train and a test set
from sklearn.model_selection import train_test_split
test_size = 0.3
seed = 2019
X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=test_size, random_state=seed)
model = best_clf
model.fit(X_train, Y_train)
results = model.score(X_test, Y_test)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
# Evaluate using Cross Validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

num_folds = 18
seed = 2019
kfold = KFold(n_splits=num_folds, random_state=seed)
model = best_clf
results = cross_val_score(model, X, y, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
# Cross Validation Classification LogLoss
kfold = KFold(n_splits=10, random_state=7)
model = best_clf
scoring = 'neg_log_loss'
results = cross_val_score(model, X, y, cv=kfold, scoring=scoring)
print("Logloss: %.3f (%.3f)" % (results.mean(), results.std()))

In [ ]:
# Evaluate using Shuffle Split Cross Validation
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

n_splits = 29
test_size = 0.33
seed = 2019
kfold = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=seed)
model = best_clf
results = cross_val_score(model, X, y, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
# from sklearn.svm import SVC

# num_folds = 14
# seed = 2019
# kfold = KFold(n_splits=num_folds, random_state=seed)
# model = SVC(C=1.0, kernel='rbf')
# results = cross_val_score(model, X, y, cv=kfold)
# print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

# Improve Performance with Ensembles

A.1 Bagged Decision Trees

In [ ]:
# Bagged Decision Trees for Classification

from sklearn.ensemble import BaggingClassifier

num_folds = 18
seed = 2019
kfold = KFold(n_splits=num_folds, random_state=seed)
num_trees = 100
model = BaggingClassifier(base_estimator=best_clf, n_estimators=num_trees, random_state=seed)
results = cross_val_score(model, X, y, cv=kfold)
print(results.mean(), results.std())

A.2 Random Forest

In [ ]:
# Random Forest Classification
from sklearn.ensemble import RandomForestClassifier

num_trees = 100
max_features = 3
kfold = KFold(n_splits=18, random_state=2019)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, X, y, cv=kfold)
print(results.mean(), results.std())

A.3 Extra Trees

In [ ]:
# Extra Trees Classification
from sklearn.ensemble import ExtraTreesClassifier

num_trees = 100
max_features = 7
kfold = KFold(n_splits=18, random_state=2019)
model = ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, X, y, cv=kfold)
print(results.mean(), results.std())

# Comparison ML packages

In [ ]:
# Load ML packages

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC

In [ ]:
# Prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', best_clf))
models.append(('NB', GaussianNB()))
models.append(('RFC', RandomForestClassifier()))
# models.append(('SVM', SVC()))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    kfold = KFold(n_splits=18, random_state=2019)
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# boxplot algorithm comparison

fig = plt.figure(figsize=(9, 9))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()